# Question 1
Réservoir de lanceur cylindrique de 3m de diamètre, 8m de longueur et 0.123mm d'épaisseur.
## Calcul de $N_x$ et $N_y$ induit par la pression $\Delta p = 300$kPa

In [1]:
import numpy as np
delta_p = 300e3 # N/m^2 = kg/m/s^2
R = 3 # m
e = 0.123e-3 # m
Nx_p = delta_p * R/2 # N/m
Ny_p = delta_p * R
print(f'Nx_p = {Nx_p / 1e3:.2f} kN/m')
print(f'Ny_p = {Ny_p / 1e3:.2f} kN/m')

Nx_p = 450.00 kN/m
Ny_p = 900.00 kN/m


## Calcul de $N_x$ causée par $F = 3600$kN

In [2]:
F = 3600e3
Nx_F = F / (2*np.pi*R)
print(f'Nx_F = {Nx_F / 1e3:.2f} kN/m')

Nx_F = 190.99 kN/m


## Calcul de $N_{xy}$ causée par $T=1200$kN$\cdot$m

In [3]:
T = 1200e3
Nxy_T = T / (np.pi*R**2)
print(f'Nxy_T = {Nxy_T / 1e3:.2f} kN/m')

Nxy_T = 42.44 kN/m


## Propositions de séquences de calculs
Il faut minimiser D16 et D26, pour ce faire:
- Il faut que le stratifié soit symétrique et équilibré ou croisé.
- Il faut éviter de grouper un trop grand nombre de plis contigus dans une même direction (jamais plus de trois ou quatre).
- Il faut un stratifié ayant au moins 4 orientations différentes $(0\degree, \pm\theta, 90\degree)$
- Il faut alterner les couches à $+\theta$ et $-\theta$ et placer la paire le plus près possible
- Il faut Placer les plis à $+45\degree$ ou $-45\degree$ en surface, puis les plis à $90\degree$, et les plis à $0\degree$.
- Si le stratifié est chargé principalement en cisaillement, les plis à $+45\degree$ ou $-45\degree$ ne devraient pas être en surface.

In [4]:
from laminate import *
from composite import *

def test_angle_combinations(combinations):
    averages = []
    angles_list = []
    for index, combination in enumerate(combinations):
        safe = False
        repetitions = 1
        D16, D26, worst_security_factor = 0, 0, 0
        lam, angles = None, None
        while not safe:
            angles = combination + f"{repetitions}" + "S"
            lam = Laminate(thetas=LaminateAngles(angles), h=0.123, composite_type=CompositeType.IM7_977_2)
            lam.add_forces_and_moments(ns=[Nx_F, 0, Nxy_T], ms=[0, 0, 0])
            pressures_tsai_hill = lam.failure_pressure_criteria(d=6, criteria=FailureCriteria.TsaiHill, p=300e3)
            security_factors = {key: pressures_tsai_hill[key]['f_s'] for key in pressures_tsai_hill.keys()}
            worst_security_factor = min(security_factors.values())
            D16 = lam.d_matrix[0, 2]
            D26 = lam.d_matrix[1, 2]
            if worst_security_factor > 3:
                safe = True
            else:
                repetitions += 1
        print(f"Combinaison {angles} = D16: {D16}, D26: {D26}")
        print(f"Nombre de plis: {len(lam.thetas)}")
        print(f"Facteur de sécurité le plus bas: {worst_security_factor}")
        averages.append((D16 + D26) / 2)
        angles_list.append(angles)
    
    best = min(averages)
    best_index = averages.index(best)
    rejected_combinations = [angle for index, angle in enumerate(angles_list) if index != best_index]
    print(f"Meilleure séquence: {angles_list[best_index]}")
    print(f"Combinaisons rejetées: {rejected_combinations}")
    return angles_list[best_index]

# "[0_3/45/90_2/-45/90_4/45/90/-45/90_2/±45]"
initial_combinations = ["[0_2/45_2/0_2/-45_2/0]", "[±45/90/0/±45/90/0]", "[0_2/±45/0_2/±45/0]", "[±45/±45/90_2/0]", "[90_2/0_2/±45]", "[±45/90/0]"]
initial_best_sequence = test_angle_combinations(initial_combinations)

Combinaison [0_2/45_2/0_2/-45_2/0]5S = D16: 281.71824499313493, D26: 281.71824499313493
Nombre de plis: 90
Facteur de sécurité le plus bas: 3.16343
Combinaison [±45/90/0/±45/90/0]3S = D16: 24.970480806209686, D26: 24.9704808062097
Nombre de plis: 48
Facteur de sécurité le plus bas: 4.06378
Combinaison [0_2/±45/0_2/±45/0]5S = D16: 70.42956124828441, D26: 70.42956124828437
Nombre de plis: 90
Facteur de sécurité le plus bas: 3.16343
Combinaison [±45/±45/90_2/0]3S = D16: 23.049674590347372, D26: 23.049674590347404
Nombre de plis: 42
Facteur de sécurité le plus bas: 3.87216
Combinaison [90_2/0_2/±45]3S = D16: 6.722821755517985, D26: 6.722821755518018
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.11159
Combinaison [±45/90/0]5S = D16: 17.60739031207094, D26: 17.607390312070947
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Meilleure séquence: [90_2/0_2/±45]3S
Combinaisons rejetées: ['[0_2/45_2/0_2/-45_2/0]5S', '[±45/90/0/±45/90/0]3S', '[0_2/±45/0_2/±45/0]5S', '[±45/±45/90

In [5]:
second_combinations = ["[90_2/0_2/±45]", "[90_3/0_2/±45]", "[90_2/0_3/±45]", "[90_2/0_2/±45_2]", "[90_2/0_2/±45/0_2]", "[90_2/0_2/±45/0_3]"]
second_best_sequence = test_angle_combinations(second_combinations)

Combinaison [90_2/0_2/±45]3S = D16: 6.722821755517985, D26: 6.722821755518018
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.11159
Combinaison [90_3/0_2/±45]3S = D16: 7.683224863449137, D26: 7.683224863449196
Nombre de plis: 42
Facteur de sécurité le plus bas: 3.78746
Combinaison [90_2/0_3/±45]3S = D16: 7.683224863449137, D26: 7.683224863449177
Nombre de plis: 42
Facteur de sécurité le plus bas: 3.27636
Combinaison [90_2/0_2/±45_2]3S = D16: 19.208062158622816, D26: 19.208062158622855
Nombre de plis: 48
Facteur de sécurité le plus bas: 4.06378
Combinaison [90_2/0_2/±45/0_2]3S = D16: 10.56443418724254, D26: 10.564434187242592
Nombre de plis: 48
Facteur de sécurité le plus bas: 3.39738
Combinaison [90_2/0_2/±45/0_3]3S = D16: 12.485240403104797, D26: 12.485240403104884
Nombre de plis: 54
Facteur de sécurité le plus bas: 3.49888
Meilleure séquence: [90_2/0_2/±45]3S
Combinaisons rejetées: ['[90_3/0_2/±45]3S', '[90_2/0_3/±45]3S', '[90_2/0_2/±45_2]3S', '[90_2/0_2/±45/0_2]3S', '[90_2/0_2

In [6]:
third_combinations = ["[90_2/0_2/±45]", "[±45_2/90_2/ 0_2]", "[±45_3/ 90_2/ 0]", "[±45/ 90_2/ 0_2]", "[±45/ 90_3/ 0_2]", "[±45/ 90_2/ 0_3]"]
third_best_sequence = test_angle_combinations(third_combinations)

Combinaison [90_2/0_2/±45]3S = D16: 6.722821755517985, D26: 6.722821755518018
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.11159
Combinaison [±45_2/90_2/ 0_2]3S = D16: 26.891287022071914, D26: 26.891287022071925
Nombre de plis: 48
Facteur de sécurité le plus bas: 4.06378
Combinaison [±45_3/ 90_2/ 0]2S = D16: 20.168465266553948, D26: 20.168465266553934
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.24132
Combinaison [±45/ 90_2/ 0_2]3S = D16: 10.564434187242535, D26: 10.564434187242544
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.11159
Combinaison [±45/ 90_3/ 0_2]3S = D16: 12.485240403104797, D26: 12.485240403104854
Nombre de plis: 42
Facteur de sécurité le plus bas: 3.78746
Combinaison [±45/ 90_2/ 0_3]3S = D16: 12.485240403104797, D26: 12.48524040310484
Nombre de plis: 42
Facteur de sécurité le plus bas: 3.27636
Meilleure séquence: [90_2/0_2/±45]3S
Combinaisons rejetées: ['[±45_2/90_2/ 0_2]3S', '[±45_3/ 90_2/ 0]2S', '[±45/ 90_2/ 0_2]3S', '[±45/ 90_3/ 0_2]3S', '[

In [7]:
fourth_combinations = ["[90_2/0_2/±45]", "[90_2/±45/0]", "[90_2/0/±45]", "[90/0/±45]", "[90/±45/0]", "[90/0/±45]"]
fourth_best_sequence = test_angle_combinations(fourth_combinations)

Combinaison [90_2/0_2/±45]3S = D16: 6.722821755517985, D26: 6.722821755518018
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.11159
Combinaison [90_2/±45/0]4S = D16: 12.165106033794459, D26: 12.165106033794489
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.61455
Combinaison [90_2/0/±45]4S = D16: 10.884568556552937, D26: 10.884568556552994
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.61455
Combinaison [90/0/±45]5S = D16: 14.40604661896709, D26: 14.406046618967103
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Combinaison [90/±45/0]5S = D16: 16.006718465519082, D26: 16.006718465519114
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Combinaison [90/0/±45]5S = D16: 14.40604661896709, D26: 14.406046618967103
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Meilleure séquence: [90_2/0_2/±45]3S
Combinaisons rejetées: ['[90_2/±45/0]4S', '[90_2/0/±45]4S', '[90/0/±45]5S', '[90/±45/0]5S', '[90/0/±45]5S']


In [8]:
def combination_generator(initial_combination: str, n: int):
    combinations = [initial_combination]
    segments = initial_combination.split("/")
    for i in range(n):
        for j in range(n):
            for k in range(n):
                new_combination = ""
                for segment in segments:
                    if '[' in segment:
                        new_segment = segment + f"_{i+1}"
                        new_combination += new_segment + "/"
                    elif ']' in segment:
                        new_segment = segment.replace("]", f"_{k+1}]")
                        new_combination += new_segment
                    else:
                        new_segment = segment + f"_{j+1}"
                        new_combination += new_segment + "/"
                combinations.append(new_combination)
    return combinations

test_90_0_45 = combination_generator("[90/0/±45]", 4)
best_sequence_of_90_0_45 = test_angle_combinations(test_90_0_45)

Combinaison [90/0/±45]5S = D16: 14.40604661896709, D26: 14.406046618967103
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Combinaison [90_1/0_1/±45_1]5S = D16: 14.40604661896709, D26: 14.406046618967103
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Combinaison [90_1/0_1/±45_2]4S = D16: 28.171824499313455, D26: 28.171824499313466
Nombre de plis: 48
Facteur de sécurité le plus bas: 3.91633
Combinaison [90_1/0_1/±45_3]3S = D16: 31.693302561727595, D26: 31.693302561727585
Nombre de plis: 48
Facteur de sécurité le plus bas: 3.79863
Combinaison [90_1/0_1/±45_4]2S = D16: 23.049674590347358, D26: 23.049674590347358
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.08624
Combinaison [90_1/0_2/±45_1]5S = D16: 17.607390312070812, D26: 17.60739031207085
Nombre de plis: 50
Facteur de sécurité le plus bas: 3.56507
Combinaison [90_1/0_2/±45_2]4S = D16: 32.013436931038044, D26: 32.013436931038086
Nombre de plis: 56
Facteur de sécurité le plus bas: 3.99583
Combinaison [90

In [9]:
print(best_sequence_of_90_0_45)
combination_90_0_45 = best_sequence_of_90_0_45.split("]")[0] + "]"
winning_combination_90_0_45 = test_angle_combinations([combination_90_0_45])

[90_4/0_3/±45_1]2S
Combinaison [90_4/0_3/±45_1]2S = D16: 3.521478062414201, D26: 3.5214780624142437
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.21960
Meilleure séquence: [90_4/0_3/±45_1]2S
Combinaisons rejetées: []


In [10]:
test_45_90_0 = combination_generator("[±45/90/0]", 4)
best_sequence_of_45_90_0 = test_angle_combinations(test_45_90_0)

Combinaison [±45/90/0]5S = D16: 17.60739031207094, D26: 17.607390312070947
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Combinaison [±45_1/90_1/0_1]5S = D16: 17.60739031207094, D26: 17.607390312070947
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.38648
Combinaison [±45_1/90_1/0_2]5S = D16: 22.409405851726586, D26: 22.409405851726618
Nombre de plis: 50
Facteur de sécurité le plus bas: 3.56507
Combinaison [±45_1/90_1/0_3]5S = D16: 27.211421391382537, D26: 27.211421391382572
Nombre de plis: 60
Facteur de sécurité le plus bas: 3.70170
Combinaison [±45_1/90_1/0_4]4S = D16: 21.128868374485222, D26: 21.128868374485272
Nombre de plis: 56
Facteur de sécurité le plus bas: 3.06196
Combinaison [±45_1/90_2/0_1]4S = D16: 14.726180988277555, D26: 14.726180988277584
Nombre de plis: 40
Facteur de sécurité le plus bas: 3.61455
Combinaison [±45_1/90_2/0_2]3S = D16: 10.564434187242535, D26: 10.564434187242544
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.11159
Combinaison [±

In [11]:
print(best_sequence_of_45_90_0)
combination_45_90_0 = best_sequence_of_45_90_0.split("]")[0] + "]"
winning_combination_45_90_0 = test_angle_combinations([combination_45_90_0])

[±45_1/90_4/0_3]2S
Combinaison [±45_1/90_4/0_3]2S = D16: 8.003359232759493, D26: 8.003359232759511
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.21960
Meilleure séquence: [±45_1/90_4/0_3]2S
Combinaisons rejetées: []


In [12]:
best_sequence_of_all = test_angle_combinations([winning_combination_90_0_45.split("]")[0] + "]", winning_combination_45_90_0.split("]")[0] + "]"])
print("Meilleure séquence de tous les tests: ", best_sequence_of_all)

Combinaison [90_4/0_3/±45_1]2S = D16: 3.521478062414201, D26: 3.5214780624142437
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.21960
Combinaison [±45_1/90_4/0_3]2S = D16: 8.003359232759493, D26: 8.003359232759511
Nombre de plis: 36
Facteur de sécurité le plus bas: 3.21960
Meilleure séquence: [90_4/0_3/±45_1]2S
Combinaisons rejetées: ['[±45_1/90_4/0_3]2S']
Meilleure séquence de tous les tests:  [90_4/0_3/±45_1]2S


# Question 2
Quand le réservoir est rempli de carburant cryogénique, la température de la surface du réservoir
en contact avec le carburant devient égale à la température du carburant. Bien qu’un gradient de
température se forme à travers l’épaisseur de la paroi, on supposera que la température est constante
et égale à $-193\degree$C. Sachant que le réservoir a été fabriqué à $100\degree$C:

## a) Déterminez quelles sont les contraintes thermiques qui se développent en service dans le réservoir. Commentez les valeurs obtenues. 

In [17]:
lam = Laminate(thetas=LaminateAngles("[90_4/0_3/±45]2S"), h=0.123, composite_type=CompositeType.IM7_977_2, delta_t=293)
lam.add_forces_and_moments(ns=[Nx_F, 0, Nxy_T], ms=[0, 0, 0])
pressures_tsai_hill = lam.failure_pressure_criteria(d=6, criteria=FailureCriteria.TsaiHill, p=300e3, verbose=True)

Thermal loads: [231.022 213.363  -0.      0.      0.     -0.   ] kN/m


## b) Ajoutez les contraintes causées par le chargement (sans le facteur de sécurité) et vérifiez si le réservoir est capable de supporter le chargement. Commentez vos résultats et discutez de votre design.

In [18]:
print(pressures_tsai_hill)

{90: {'p': 300000.0, 'f_s': 2.10008}, 0: {'p': 300000.0, 'f_s': 2.16571}, 45: {'p': 300000.0, 'f_s': 2.00817}, -45: {'p': 300000.0, 'f_s': 2.28145}}


On remarque que toutes les couches tombent en bas du facteur de sécurité recommandé de 3. Le facteur de sécurité le plus bas est celui de la couche de $45\degree$ qui est d'environ $2.01$.

In [19]:
lam_no_temp = Laminate(thetas=LaminateAngles("[90_4/0_3/±45]2S"), h=0.123, composite_type=CompositeType.IM7_977_2)
lam_no_temp.add_forces_and_moments(ns=[Nx_F, 0, Nxy_T], ms=[0, 0, 0])
pressures_tsai_hill_no_temp = lam_no_temp.failure_pressure_criteria(d=6, criteria=FailureCriteria.TsaiHill, p=300e3)
print(pressures_tsai_hill_no_temp)

{90: {'p': 300000.0, 'f_s': 3.49999}, 0: {'p': 300000.0, 'f_s': 3.29862}, 45: {'p': 300000.0, 'f_s': 3.59958}, -45: {'p': 300000.0, 'f_s': 3.21960}}


In [26]:
lam_45 = Laminate(thetas=LaminateAngles("[±45/90_4/0_3]2S"), h=0.123, composite_type=CompositeType.IM7_977_2)
lam_45.add_forces_and_moments(ns=[Nx_F, 0, Nxy_T], ms=[0, 0, 0])
pressures_tsai_hill_no_temp_45 = lam_45.failure_pressure_criteria(d=6, criteria=FailureCriteria.TsaiHill, p=300e3)
print(pressures_tsai_hill_no_temp)

{90: {'p': 300000.0, 'f_s': 3.49999}, 0: {'p': 300000.0, 'f_s': 3.29862}, 45: {'p': 300000.0, 'f_s': 3.59958}, -45: {'p': 300000.0, 'f_s': 3.21960}}


In [27]:
lam_45_w_temp = Laminate(thetas=LaminateAngles("[±45/90_4/0_3]2S"), h=0.123, composite_type=CompositeType.IM7_977_2, delta_t=293)
lam_45_w_temp.add_forces_and_moments(ns=[Nx_F, 0, Nxy_T], ms=[0, 0, 0])
pressures_tsai_hill_w_temp_45 = lam_45_w_temp.failure_pressure_criteria(d=6, criteria=FailureCriteria.TsaiHill, p=300e3, verbose=True)
print(pressures_tsai_hill_w_temp_45)

Thermal loads: [231.022 213.363  -0.      0.      0.     -0.   ] kN/m
{45: {'p': 300000.0, 'f_s': 2.00817}, -45: {'p': 300000.0, 'f_s': 2.28145}, 90: {'p': 300000.0, 'f_s': 2.10008}, 0: {'p': 300000.0, 'f_s': 2.16571}}
